In [8]:
import pickle
X_acl,X_ppg,y,y_participant = pickle.load(open('../data/tabular_data.p','rb'))

In [9]:
from sklearn.preprocessing import RobustScaler,MinMaxScaler

In [10]:
for k in range(X_ppg.shape[0]):
    X_ppg[k] = MinMaxScaler().fit_transform(X_ppg[k])

In [11]:
import numpy as np
X_ppg = X_ppg[:,np.arange(0,512,2),:]

X_acl = np.concatenate([X_ppg,X_acl],axis=-1)

y = X_ppg[:,:,0].reshape(-1,256,1)
X_acl = X_acl[:,:,:1]

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional
from keras.models import Model, Sequential

In [13]:
logo = LeaveOneGroupOut()
logo = LeavePGroupsOut(n_groups=5)
for train_index, test_index in logo.split(X_acl, y, y_participant.reshape(-1)):
    train_x, test_x = X_acl[train_index], X_acl[test_index]
    train_y, test_y = y[train_index], y[test_index]
    train_participant, test_participant = y_participant[train_index], y_participant[test_index]
    break
train_x.shape,train_y.shape
pickle.dump([train_participant,test_participant],open('../data/participant_split_for_BRDAE.p','wb'))

In [14]:
from joblib import Parallel,delayed

def random_noise(x):
    return x+(1/3)*np.random.normal(0,1,x.shape)

def sloping_noise(x):
    x = x.copy()
    def corrupt_single(xx):
        xx_temp = np.random.uniform(-1,1)
        xx_temp = np.array([xx_temp]*256).reshape(256,1)
        index = np.arange(0,256,1).reshape(256,1)
        return xx+(4/250)*np.multiply(index,xx_temp)

    for i in range(x.shape[0]):
        x[i] = corrupt_single(x[i])
    return x


def saturation_noise(x):
    x = x.copy()
    for i in range(x.shape[0]):
        low = np.random.randint(0,216)
        high = np.random.randint(low,min(256,np.random.randint(low+1,low+51)))
        x[i][low:high,0] = np.random.randint(0,2)
    return x

def generate_combinations(x):
    functions = [random_noise,saturation_noise,sloping_noise]
    indicator = np.random.randint(0,2,3)
#     indicator = [1,1,1]
    for j,i in enumerate(indicator):
        if i==1:
            x = functions[j](x)
    return x


def generate_corrupt_all(train_x,n):
    X = Parallel(n_jobs=20,verbose=2)(delayed(generate_combinations)(train_x.copy()) for i in range(n))
    return np.concatenate(X)

n = 50
train_y = np.concatenate([train_x]*n)
train_x = generate_corrupt_all(train_x,n=n)
 


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  37 out of  50 | elapsed:    5.3s remaining:    1.9s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:    6.5s finished


In [15]:
train_x, val_x, train_y, val_y = train_test_split(train_x,train_y,test_size = 0.2,random_state=42)

(train_x.shape, train_y.shape), (val_x.shape, val_y.shape),(test_x.shape,test_y.shape)

(((1140480, 256, 1), (1140480, 256, 1)),
 ((285120, 256, 1), (285120, 256, 1)),
 ((14851, 256, 1), (14851, 256, 1)))

In [16]:
def get_model(input_shape=(256,1),act='tanh',loss="mae",opt='adam',n_classes=1):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='tanh',kernel_initializer='normal',padding='same'))
    model.add(MaxPool1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,20,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPool1D(5))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(50,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPool1D(2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(20,activation='relu'))
    model.add(Reshape((10,2)))
    model.add(Conv1D(100,2,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(256,2,input_shape=input_shape,activation='tanh',kernel_initializer='normal',padding='same'))
    model.add(Reshape((256,10)))
    model.add(TimeDistributed(Dense(1,activation='sigmoid')))
#     model.add(Dense(64,activation='relu'))
#     model.add(Dense(128,activation='tanh'))
#     model.add(Dense(256,activation='sigmoid'))
#     model.add(Reshape((256,1)))
    
    
#     model.add(Bidirectional(LSTM(100,return_sequences=True,activation='tanh',kernel_initializer='normal')))
#     model.add(TimeDistributed(Dense(1,activation='')))
    model.compile(loss=loss,optimizer=opt)
    return model

def get_model_LSTM(input_shape=(256,1),act='tanh',loss="mse",opt='adam',n_classes=1):
    model = Sequential()
    model.add(GaussianNoise(stddev=1))
    model.add(Bidirectional(LSTM(40,return_sequences=True,
                      activation='tanh',input_shape=input_shape)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss=loss,optimizer=opt)
    return model

In [17]:
model = get_model_LSTM()

In [21]:
from keras.models import load_model
filepath = '../model_files/BRDAE_final_1.hdf5'
model = load_model(filepath)

In [22]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=200, batch_size=1000,
          callbacks=callbacks_list,shuffle=True)

Train on 1140480 samples, validate on 285120 samples
Epoch 1/200
1140480/1140480 [==============================] - 276s 242us/step - loss: 0.0344 - val_loss: 0.0249

Epoch 00001: val_loss improved from inf to 0.02487, saving model to ../model_files/BRDAE_final_1.hdf5
Epoch 2/200
1140480/1140480 [==============================] - 276s 242us/step - loss: 0.0343 - val_loss: 0.0245

Epoch 00002: val_loss improved from 0.02487 to 0.02450, saving model to ../model_files/BRDAE_final_1.hdf5
Epoch 3/200
1140480/1140480 [==============================] - 276s 242us/step - loss: 0.0343 - val_loss: 0.0242

Epoch 00003: val_loss improved from 0.02450 to 0.02424, saving model to ../model_files/BRDAE_final_1.hdf5
Epoch 4/200
1140000/1140480 [============================>.] - ETA: 0s - loss: 0.0343

KeyboardInterrupt: 

In [ ]:
filepath = '../model_files/BRDAE_final_1.hdf5'
model = load_model(filepath)

In [ ]:
test_y_pred = model.predict(test_x)

In [ ]:
for k in range(test_y_pred.shape[0]):
    test_y_pred[k]  = MinMaxScaler().fit_transform(test_y_pred[k])

In [ ]:
%matplotlib inline
for i in range(1,100,4):
    
    fig,ax = plt.subplots(4,1,figsize=(10,7))
    ax[0].plot(test_y[i])
    ax[0].plot(test_y_pred[i],'r')
    ax[1].plot(test_y[i+1])
    ax[1].plot(test_y_pred[i+1],'r')
    ax[2].plot(test_y[i+2])
    ax[2].plot(test_y_pred[i+2],'r')
    ax[3].plot(test_y[i+3])
    ax[3].plot(test_y_pred[i+3],'r')
    
    plt.show()